## Preparation and QC of UKB participants

In [61]:
import os
import sys
import pandas as pd
import subprocess
rootdir = '/home/preclineu/ramcir/Desktop/Diffusion/Subs/1000432/dMRI/dMRI'
#os.path.exists(rootdir)

In [50]:
def prep_sub(rootdir):
    #Create symbolic link for data_ud with the name data
    #This is necessary for running 'quad' since all other eddy outputs are called data.eddy.-
    data_ud = os.path.join(rootdir, 'data_ud.nii.gz')
    data = os.path.join(rootdir, 'data.nii.gz')
    if os.path.islink(data) == False:
        os.symlink(data_ud, data)
    else:
        pass

    #Write command for creating the b0 mask and run the bash process
    fslroi_cmd = 'fslroi data.nii.gz b0_data.nii.gz 0 1'
    fslbet_cmd = 'bet b0_data.nii.gz nodif -f 0.1 -g 0 -m -n'
    prep_cmd = "cd " + rootdir + " && " + fslroi_cmd + " && " + fslbet_cmd 
    subprocess.run(prep_cmd, shell = True)

    #Create aquision parameters file and index file
    #The acquisition parameters file is based on dataset acq documentation
    #i.e. AP dir = 0 1 0, PA dir = 0 -1 0, Echo spacing = 0.67 ms, EPI factor = 104 (for UKB)
    with open(os.path.join(rootdir,'acqparams.txt'), 'w') as acqparams:
        acqparams.write('0 1 0 0.069\n0 -1 0 0.069')

    #The index file indicated the direction in which each volume was aquired (based on aqcparams.txt)
    #count number of volumes present in data
    with open(os.path.join(rootdir,'bvals'), 'r') as bvals:
        data = bvals.read()
        vols = data.split()
        no_vols = len(vols)
    with open(os.path.join(rootdir,'index.txt'), 'w') as index:
        for vol in range(0,no_vols):    
            index.write('1 ')
        

In [51]:
prep_sub(rootdir)

In [59]:
qc_dir = os.path.join(rootdir,'data.qc') 

def quad_sub(rootdir):  
    #check to see if quad has already been run
    if os.path.exists(qc_dir) == False:
        #compose and run quad command
        quad_cmd = 'eddy_quad data  -idx index.txt -par acqparams.txt -m nodif_mask -b bvals -g bvecs'
        prep_cmd = "cd " + rootdir + " && " + quad_cmd
        process = subprocess.run(prep_cmd, shell = True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        #save the quad folder's path to a txt file, this will be needed for squad
        with open(os.path.join(rootdir,'quad_folders.txt'), 'w') as quad_folders:
            quad_folders.write(qc_dir)
    else:
        print('QUAD already exists')     

In [60]:
quad_sub(rootdir)

QUAD already exists


In [ ]:
def quad_sub(rootdir):    
    if os.path.exists(qc_dir) == False:
        quad_cmd = 'eddy_quad data  -idx index.txt -par acqparams.txt -m nodif_mask -b bvals -g bvecs'
        quad_folders = rootdir + 'quad_folders.txt'
        prep_cmd = "cd " + rootdir + " && " + quad_cmd
        process = subprocess.run(prep_cmd, shell = True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    else:
        print('qc already there')